In [6]:
import sys
sys.path.append('F:/time_step/OfflineRL_FactoredActions')
from types import SimpleNamespace

import numpy as np

from RL_mimic_sepsis.d_BCQ.src.model import BCQ
from RL_mimic_sepsis.e_fair_comparison.cross_evaluation import evaluate_cross_dt, load_dataset_t2, get_ckpt_path
from RL_mimic_sepsis.e_fair_comparison.cross_evaluation import load_dataset_t2
from RL_mimic_sepsis.d_BCQ.src.data import remap_rewards


In [7]:
t1 = 1
t2 = 1  
ckpt_path = get_ckpt_path(t1)
model = BCQ.load_from_checkpoint(ckpt_path)
model.eval()

BCQ(
  (Q): BCQ_Net(
    (q): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=25, bias=True)
    )
  )
  (Q_target): BCQ_Net(
    (q): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=25, bias=True)
    )
  )
)

In [8]:

def get_test_data_path(timestep):
    action_space = 'NormThreshold'  # or 'Quantile' depending on the model
    return (rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data'
            rf'\data_as{action_space}_dt{timestep}h\episodes+encoded_state+knn_pibs_final\test_data.pt')


data_path = get_test_data_path(t2)
dataset_t2 = load_dataset_t2(data_path, t2, action_space='NormThreshold')


Episodic Buffer loaded with 2757 episides.


In [9]:
dataset_t2.reward = remap_rewards(dataset_t2.reward, SimpleNamespace(R_immed=0.0, R_death=0.0, R_disch=100.0))

In [10]:
wis_est, wis_ess = evaluate_cross_dt(model, dataset_t2, t1, t2, eps=0.1, clipping=1.438, pib = 'policy')
print(f"Cross-Δt OPE (t1={t1}h policy on t2={t2}h data): WIS = {wis_est:.4f}, ESS = {wis_ess:.2f}")
# 90, 37.08

Cross-Δt OPE (t1=1h policy on t2=1h data): WIS = 89.8486, ESS = 58.11
